In [ ]:
import os
import glob
import wandb
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Dict

import tensorflow as tf
import tensorflow.keras as keras
from alibi.models.tensorflow.autoencoder import AE
from alibi.models.tensorflow.actor_critic import Actor, Critic
from alibi.models.tensorflow.cfrl_models import MNISTEncoder, MNISTDecoder, MNISTClassifier
from alibi.explainers.cfrl_base import CounterfactualRLBase, ExperienceCallback, TrainingCallback

%load_ext autoreload
%autoreload 2

## Load MNIST dataset

In [ ]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
BATCH_SIZE = 64

# define trainset
trainset_classifier = tf.data.Dataset.from_tensor_slices((
    np.expand_dims(x_train, axis=-1).astype(np.float) / 255., y_train))
trainset_classifier = trainset_classifier.shuffle(buffer_size=1024).batch(BATCH_SIZE)

# define testset
testset_classifier = tf.data.Dataset.from_tensor_slices(
    (np.expand_dims(x_test, axis=-1).astype(np.float) / 255., y_test))
testset_classifier = testset_classifier.shuffle(buffer_size=1024).batch(BATCH_SIZE)

### Define and train classifier

In [ ]:
# number of classes
num_classes = 10

# define classifier
classifier_path = "tensorflow/classifier/classifier_mnist.tf"
classifier = MNISTClassifier(output_dim=num_classes)

# define optimizer and loss
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# fit and compile
classifier.compile(optimizer=optimizer, 
                   loss=loss,
                   metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


if len(glob.glob(classifier_path + "*")) == 0:
    # fit and save the classifier
    classifier.fit(trainset_classifier, epochs=5)
    classifier.save_weights(classifier_path)
else:
    # load the classifier
    classifier.load_weights(classifier_path).expect_partial()

In [ ]:
# evaluate the classifier
classifier.evaluate(testset_classifier)

### Define and train AE

In [ ]:
# define trainset
trainset_ae = tf.data.Dataset.from_tensor_slices(
    np.expand_dims(x_train, axis=-1).astype(np.float) / 255.
)
trainset_ae = trainset_ae.map(lambda x: (x, x))
trainset_ae = trainset_ae.shuffle(buffer_size=1024).batch(BATCH_SIZE)

# define testset
testset_ae = tf.data.Dataset.from_tensor_slices(
    np.expand_dims(x_test, axis=-1).astype(np.float) / 255.
)
testset_ae = testset_ae.map(lambda x: (x, x))
testset_ae = testset_ae.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [ ]:
latent_dim = 64

In [ ]:
# define autoencoder
ae_path = "tensorflow/autoencoder/autoencoder_mnist.tf"
ae = AE(encoder=MNISTEncoder(latent_dim=latent_dim),
        decoder=MNISTDecoder())

# define optimizer and loss
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss = keras.losses.BinaryCrossentropy(from_logits=False)

# compile autoencoder
ae.compile(optimizer=optimizer, loss=loss)

if len(glob.glob(ae_path + "*")) == 0:
    # fit and save autoencoder
    ae.fit(trainset_ae, epochs=50)
    ae.save_weights(ae_path)
else:
    # load the model
    ae.load_weights(ae_path).expect_partial()

### Test the autoencoder

In [ ]:
# get some random samples from test
num_samples = 5
np.random.seed(0)
indices = np.random.choice(x_test.shape[0], num_samples)
inputs = [x_test[i].reshape(1, 28, 28, 1) for i in indices]
inputs = np.concatenate(inputs, axis=0) / 255.

# pass samples through the autoencoder
inputs_hat = ae(inputs).numpy()

In [ ]:
# plot inputs and reconstructions
plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(2, num_samples, figsize=(25, 10))

for i in range(num_samples):
    ax[0][i].imshow(inputs[i])
    ax[1][i].imshow(inputs_hat[i])
    
    if i == 0:
        ax[0][i].set_ylabel("x")
        ax[1][i].set_ylabel("x_hat")
        

### Counterfactual RL

In [ ]:
# define prediction function
def predict_func(X: np.ndarray):
    y = tf.argmax(classifier(X), axis=1).numpy()
    return y

In [ ]:
class RewardCallback(ExperienceCallback):
    def __call__(self,
                 step: int, 
                 model: CounterfactualRLBase, 
                 sample: Dict[str, np.ndarray]):
        if step % 100 != 0:
            return
        
        # get the counterfactual and target
        x_cf = sample["x_cf"]
        y_t = sample["y_t"]
        
        # get prediction label
        y_m_cf = predict_func(x_cf)
        
        # compute reward
        reward = np.mean(model.params["reward_func"](y_m_cf, y_t))
        wandb.log({"reward": reward})
        
class DisplayImgsCallback(ExperienceCallback):
    def __call__(self,
                 step:int,
                 model: CounterfactualRLBase,
                 sample: Dict[str, np.ndarray]):
        if step % 100 != 0:
            return
        
        # defie number of samples to be displayed
        num_sample = 5
        
        x = sample["x"][:num_samples]        # input instance
        x_cf = sample["x_cf"][:num_samples]  # counterfactual
        diff = np.abs(x - x_cf)              # differences
        
        y_m = sample["y_m"][:num_samples].astype(int)   # input labels
        y_t = sample["y_t"][:num_samples].astype(int)   # target labels
        y_m_cf = predict_func(x_cf).astype(int)         # counterfactual labels
        
        # concatentate images
        x = np.concatenate(x, axis=1)
        x_cf = np.concatenate(x_cf, axis=1)
        diff = np.concatenate(diff, axis=1)
        
        # construct image
        img = np.concatenate([x, x_cf, diff], axis=0)
            
        # construct caption
        caption = ""
        caption += "Input:\t%s\n" % str(list(y_m))
        caption += "Target:\t%s\n" % str(list(y_t))
        caption += "Predicted:\t%s\n" % str(list(y_m_cf))
        
        # log image
        wandb.log({"samples": wandb.Image(img, caption=caption)})


class DisplayLossCallback(TrainingCallback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRLBase,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # log training losses
        if (step + update) % 100 == 0:
            wandb.log(losses)

In [ ]:
# define ddpg
explainer = CounterfactualRLBase(predict_func=predict_func,
                                 ae=ae,
                                 latent_dim=latent_dim,
                                 coeff_sparsity=7.5,
                                 coeff_consistency=0,
                                 num_classes=10,
                                 backend="tensorflow",
                                 experience_callbacks=[RewardCallback(), DisplayImgsCallback()],
                                 train_callbacks=[DisplayLossCallback()],
                                 train_steps=10000)

In [ ]:
# initialize wandb
wandb_project = "MNIST CounterfactualRL"
wandb.init(project=wandb_project)

# add channel and normalize
x = np.expand_dims(x_train, axis=-1) / 255.

# fit the explainer
explainer.fit(x=x)

# close wandb
wandb.finish()

In [ ]:
i = 25
x_cf = explainer.explain(x[i:i+1], y_t=np.array([0]))

plt.imshow(x[i])
plt.title(f"Label: {predict_func(x[i:i+1]).item()}")
plt.show()

plt.imshow(x_cf[0])
plt.title(f"Label: {predict_func(x_cf).item()}")
plt.show()